# 2. Named Entity Recognition With Conditional Random Fields(CRF) In Python

저번 포스트에서는 해당 단어만의 특징들을 이용해 NER을 수행해보았습니다. 하지만, 결과는 매우 좋지 않았죠. 

이번 모델에서는 맥락(context)과 구조를 활용해 NER을 수행합니다. 

사용할 알고리즘은 : **Conditional Random Field(CRF)**입니다. 

2018년 네이버와 창원대가 함께 개최한 NLP Challenge;NER 에서도 베이스라인 모델로 Bidiriectional RNN + CRF 의 조합을 사용했죠. 

CRF는 앞으로의 포스트에도 등장할 예정이니 열심히 학습해봅시다!


앞으로의 포스트는 다음과 같습니다:
1. Introduction To NER
2. NER With CRF 
3. NER with LSTM
4. Sequence Tagging With A LSTM-CRF
...

## Conditional Random Field (CRF)
 
 Conditional Random Field(CRF)는 RNN의 등장 이전에 시퀀스 데이터를 처리하는 가장 대표적인 알고리즘입니다. 

 CRF를 한 문장으로 정의하면 다음과 같습니다:
 > Sequential labeling 을 위하여 potential functions 을 이용하는 softmax regression

Sequential Labeling이 무엇인지, Potential function이 어떤 함수인지는 **이론** 포스트를 참조하세요!

간단하게만 설명하면: 
- sequential labeling은 시퀀스가 입력으로 들어갔을 때, 출력 레이블도 시퀀스로 나오는 것을 의미합니다. 결국은 분류 모델인 것이죠. 

- CRF의 potential function은 시퀀스 데이터를 벡터로 표현합니다. 예를 들어, 띄어쓰기가 필요한 부분은 1, 아닌 부분은 0으로 표현할 수 있죠. 

- Softmax Regression은 벡터 x에 대해서 label Y를 출력하는 함수입니다. 

CRF를 사용하는 이유는 간단합니다. CRF는 앞, 뒤의 단어로 이뤄진 **문맥**을 이용하여 labeling을 합니다. 이에 따라 코퍼스에 직접적으로 등장하지 않은 단어들에 대한 대처 능력이 상승합니다! 마치 Word2vec과 같은 이치네요!

***CRF can learn context!*** 

그렇다면, 문맥을 고려하지 않고 해당 단어만을 살펴봤던 NER(Named Entity Recognition)이 CRF를 적용했을 때, 얼마나 성능이 올라가는지 확인해봅시다. 





## Data Load

In [0]:
import pandas as pd 
import numpy as np

data = pd.read_csv("/content/ner_dataset.csv", encoding='latin1')

In [0]:
# 빈 값을 앞 데이터로부터 채우기
# https://ordo.tistory.com/59

data = data.fillna(method = "ffill")

In [0]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [0]:
words = list(set(data["Word"].values))
n_words = len(words)

print(n_words)

35178


data.tail()과 n_words를 통해 알아낸 데이터의 특징:
- 문장의 개수: 46,959개
- 단어 개수 : 35,178개

그렇다면 이제 데이터를 [문장] : [POS] : [NER]의 형태로 만들어봅시다. 

지난 포스트와는 달리 이번엔 pandas 라이브러리에서 제공하는 **groupby()** 메서드를 사용합니다. 

Python pandas의 groupby() 연산자는 그룹별로 데이터를 집계하고 요약하는데 아주 편리한 기능을 제공합니다. 

전체 데이터를 그룹 별로 나누고 (split), 각 그룹별로 집계함수를 적용(apply) 한후, 그룹별 집계 결과를 하나로 합치는(combine) 단계를 거치게 됩니다. (Split => Apply function => Combine)

자세한 내용은 밑을 참고하세요!

출처: https://rfriend.tistory.com/383 [R, Python 분석과 프로그래밍의 친구 (by R Friend)]

본 포스트에서는 문장의 번호를 기준으로 나머지 데이터들의 모양을 바꾸어보겠습니다. 

In [0]:
class GetSentencePair(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False

        agg_func = lambda s: [(w,p,t) for w,p,t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        
        self.group = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.group]

    def get_next(self):
        try: 
            s = self.group["Sentence: {}".format(self.n_next)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
GetSent = GetSentencePair(data)
sent = GetSent.get_next()

In [0]:
sentences = GetSent.sentences

In [0]:
print(sentences[0])

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


## Features Collection

이번 포스트에서는 맥락을 이용한다는 말을 했습니다. 여기서의 맥락은 앞, 뒤의 단어를 의미합니다. 

이번에는 단어가 들어오면 이를 피쳐로 바꾸어주는 함수를 정의할 것입니다. 
list of (word, pos, tag) 형식의 sentence 가 입력되면 이를 feature 로 변환해 넘겨줍니다. 각 단어의 피쳐를 뽑아내는 함수를 ***WordFeatures***를 통해 구현하겠습니다.  또한, 문장을 인코딩하는 함수를  ***SentFeatures*** 라고 칭하겠습니다. 
우리가 원하는 label인 개체명 tag는 ***SentTag***로 뽑아냅니다. 


피쳐의 수집 대상이 되는 단어들은 다음의 3가지 입니다. 
 > w-1, w, w+1

수집할 피쳐의 목록은 다음과 같습니다:
- word.istitle() : 대문자로 시작
- word.islower() : 모든 문자 소문자
- word.isupper() : 모든 문자 대문자
- len(word) : 단어 길이
- pos : 품사 전체분류
- pos[:2] : 품사 대분류

해당 피쳐들은 밑에서 소개할 사이킷런의 CRF 구현 예제에서 선정한 목록을 참고했습니다. 

각 문장의 처음에는 시작지점을 의미하는 토근 **"BOS"**(Beginning of the sentence), 
끝에는 종료지점을 의미하는 토큰 **"EOS"**(End of the sentence)를 추가합니다. 





In [0]:
# 단어의 피쳐 뽑아내기

def WordFeatures(sent, i):
    word = sent[i][0]
    pos = sent[i][1]

    # 대상 단어들에게서 뽑아낼 피쳐
    features = {
        'istitle': word.istitle(),
        'islower': word.islower(), 
        'isupper': word.isupper(),
        'length': len(word), 
        'pos': pos, 
        'pos[:2]': pos[:2]
    }

    if i>0:
        context = sent[i-1][0]
        posCxt = sent[i-1][1]

        features.update({
            '-1:istitle': context.istitle(),
            '-1:islower': context.islower(),
            '-1:isupper': context.isupper(), 
            '-1:length': len(context), 
            '-1:pos': posCxt, 
            '-1:pos[:2]':posCxt[:2]
        }) 

    else: 
        features['BOS']=True

    if i< len(sent)-1:
        context = sent[i+1][0]
        posCxt = sent[i+1][1]

        features.update({
            '+1:istitle': context.istitle(),
            '+1:islower': context.islower(),
            '+1:isupper': context.isupper(), 
            '+1:length': len(context), 
            '+1:pos': posCxt, 
            '+1:pos[:2]':posCxt[:2]
        }) 

    else: 
        features['EOS']=True

    return features

In [0]:
# 문장 인코딩
def SentFeatures(sent):
    return [WordFeatures(sent, i) for i in range(len(sent))]

# 개체명 레이블
def SentTag(sent):
    return [tag for word, pos, tag in sent]

In [0]:
X = [SentFeatures(s) for s in sentences]
y = [SentTag(s) for s in sentences]

In [0]:
X[0][0]

{'+1:islower': True,
 '+1:istitle': False,
 '+1:isupper': False,
 '+1:length': 2,
 '+1:pos': 'IN',
 '+1:pos[:2]': 'IN',
 'BOS': True,
 'islower': False,
 'istitle': True,
 'isupper': False,
 'length': 9,
 'pos': 'NNS',
 'pos[:2]': 'NN'}

In [0]:
X_train = X[:40000]
X_test = X[40000:]

y_train = y[:40000]
y_test = y[40000:]

## CRF model 적용하기

CRF를 직접 구현하기보다는, 사이킷런(sklearn-crfsuite)에서 제공하는 것을 사용하겠습니다. 

자세한 내용과 간단한 튜토리얼은 [다음](https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb)을 참고하세요. 위의 코드도 이를 참고했습니다. 





In [0]:
! pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 7.0MB/s 


In [0]:
from sklearn_crfsuite import CRF

crf = CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

마지막으로 **5-fold Cross Validation(교차 검증)**을 실행하도록 하겠습니다. 

해당 검증은 말 그대로 K개의 fold를 만들어서 진행하는 교차 검증 방법으로, 데이터의 크기가 적은 셋에 대한 정확도를 향상시키기 위한 목적으로 사용됩니다. 

일반적인 train, validation, test로 데이터를 나누는 방식은 데이터셋의 크기가 작은 경우 치명적이기 때문이죠. 

자세한 내용은 [이곳](https://nonmeyet.tistory.com/entry/KFold-Cross-Validation%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D-%EC%A0%95%EC%9D%98-%EB%B0%8F-%EC%84%A4%EB%AA%85)을 참고하세요. 






In [0]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report

pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


## Model Evaluation

In [0]:
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       111
       B-eve       0.25      0.03      0.06        89
       B-geo       0.62      0.82      0.71      7695
       B-gpe       0.78      0.77      0.78      3597
       B-nat       0.00      0.00      0.00        53
       B-org       0.64      0.46      0.53      4050
       B-per       0.67      0.67      0.67      3568
       B-tim       0.73      0.42      0.53      4099
       I-art       0.00      0.00      0.00        65
       I-eve       0.29      0.10      0.15        71
       I-geo       0.54      0.48      0.51      1574
       I-gpe       0.40      0.03      0.05        68
       I-nat       0.00      0.00      0.00        22
       I-org       0.57      0.59      0.58      3274
       I-per       0.67      0.84      0.75      3734
       I-tim       0.76      0.38      0.51      1236
           O       0.98      0.99      0.99    184276

    accuracy              

In [0]:
crf.fit(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)